# Scraper reviews Meli

In [ ]:
import requests
import time
import json
import csv
from tqdm import tqdm
from bs4 import BeautifulSoup

In [ ]:
# Function to get all product link
def getLinks(product_name):
    url = 'https://lista.mercadolivre.com.br/'
    links = []
    response = requests.get(url+product_name)
    bs = BeautifulSoup(response.content, 'html.parser')
    total_page = int(bs.
              find_all('li', {'class':'andes-pagination__page-count'})[0].
              get_text().
              replace('de ','')
              )
    
    for page in tqdm(range(total_page-1)):
        links.append(bs.find_all('a',{'class':'ui-search-item__group__element shops__items-group-details ui-search-link'}))
        next_page = bs.find_all('a',{'title':'Seguinte'})
        resp = requests.get(next_page[0]['href'])
        bs = BeautifulSoup(resp.content,'html.parser')
        time.sleep(5)
    return links

In [ ]:
# Function to get all reviews by one product
def get_reviews_page(url_request):
    n = 1
    while True:
        response = requests.get(url_request+str(n)+'&x-is-webview=false')
        if response.status_code != 400: 
            n += 1
        else:
            break
    response = requests.get(url_request+str(n-1)+'&x-is-webview=false')
    data = json.loads(response.text)
    reviews = []
    for index in range(len(data['reviews'])):
        reviews.append(data['reviews'][index]['comment']['content']['text'])
    return reviews

In [ ]:
# Function to select the correct product id
def get_id_product(product):
    if product['href'].split('.')[0] == 'https://www':
        return product['href'].split('/')[-1].split('?')[0]
    elif product['href'].split('.')[0] == 'https://produto':
        return product['href'].split('/')[3].split('-')[0] + product['href'].split('/')[3].split('-')[1]
    return None

In [ ]:
# Function to get all reviews
def get_all_reviews(products_links):
    total_reviews = []
    for page in tqdm(products_links):
        product_ids = [get_id_product(product) for product in page]
        product_ids = [p_id for p_id in product_ids if p_id is not None]
        
        url_requests = [f'https://www.mercadolivre.com.br/noindex/catalog/reviews/{p_id}/search?objectId={p_id}&siteId=MLB&isItem=false&limit=' for p_id in product_ids]

        reviews = [get_reviews_page(url) for url in url_requests]
        total_reviews.extend(reviews)
    return total_reviews


In [ ]:
products_links = getLinks('tv-smart')

In [ ]:
reviews = get_all_reviews(products_link[:15])
# Open a CSV file for writing
with open('reviews-15-all.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row in the list as a new line in the CSV file
    for row in reviews:
        writer.writerow(row)

# End